In [9]:
import gate
import numpy as np

init = [1, 0, 0, 0]
print(gate.gate2['CX'] @ (np.kron(gate.gate1['H'],gate.gate1['I']) @ init))

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [10]:
for i in range(0, 10):
    k = np.kron(gate.gate1['I'],gate.gate1['H'])


In [ ]:
for i in range(0, 1000000):
    k = gate.gate2['IH']

In [27]:
def kron_A_I(A, N):
    out = np.zeros((N*N, N*N),dtype=A.dtype)
    for i in range(N):
        for j in range(N):
            value = A[i,j]
            out[i*2: i*2 + 2,j*2:j*2 + 2] = np.array([[value, 0], [0, value]])
    return out

In [68]:
A = np.random.rand(4, 4)
print(A)

[[0.08438087 0.41107181 0.04538587 0.95188399]
 [0.1705832  0.77136947 0.69109793 0.58784028]
 [0.29480806 0.77784422 0.59064393 0.70858758]
 [0.11067381 0.13596001 0.74064147 0.29661303]]


In [76]:
import numpy as np

def kron_A_X(A):
    N = A.shape[0]
    out = np.zeros((N*2, N*2),dtype=A.dtype)
    for i in range(N):
        for j in range(N):
            value = A[i,j]
            out[i*2: i*2 + 2,j*2:j*2 + 2] = np.array([[0, value], [value, 0]])
    return out


A = np.random.rand(3, 3)
result = kron_A_X(A)
print(result)


[[0.         0.48770209 0.         0.89811611 0.         0.41298587]
 [0.48770209 0.         0.89811611 0.         0.41298587 0.        ]
 [0.         0.69194952 0.         0.1066459  0.         0.41532906]
 [0.69194952 0.         0.1066459  0.         0.41532906 0.        ]
 [0.         0.49388755 0.         0.87396303 0.         0.37631692]
 [0.49388755 0.         0.87396303 0.         0.37631692 0.        ]]


In [78]:
np.allclose(np.kron(A, np.array([[0, 1], [1, 0]])), result)

True

In [88]:
A = np.random.rand(200, 200)
B = np.random.rand(100, 100)

In [90]:
for i in range(1000):
    k = np.kron(np.array([[0, 1], [1, 0]]), A)
    print(k)

[[0.         0.         0.         ... 0.72134186 0.7619806  0.3352845 ]
 [0.         0.         0.         ... 0.84433912 0.3835606  0.49107581]
 [0.         0.         0.         ... 0.6580181  0.87852711 0.28336673]
 ...
 [0.46749195 0.74126294 0.40401744 ... 0.         0.         0.        ]
 [0.82417522 0.89611517 0.34073584 ... 0.         0.         0.        ]
 [0.66964531 0.35287366 0.87009926 ... 0.         0.         0.        ]]
[[0.         0.         0.         ... 0.72134186 0.7619806  0.3352845 ]
 [0.         0.         0.         ... 0.84433912 0.3835606  0.49107581]
 [0.         0.         0.         ... 0.6580181  0.87852711 0.28336673]
 ...
 [0.46749195 0.74126294 0.40401744 ... 0.         0.         0.        ]
 [0.82417522 0.89611517 0.34073584 ... 0.         0.         0.        ]
 [0.66964531 0.35287366 0.87009926 ... 0.         0.         0.        ]]
[[0.         0.         0.         ... 0.72134186 0.7619806  0.3352845 ]
 [0.         0.         0.         ... 

In [81]:
for i in range(100):
    k = kron_X_A(A)

In [7]:
from string import ascii_lowercase as letters
from functools import reduce

def mv_kron(matrices, y):
    '''
    Compute product of vector and Kronecker-structured matrix
    via brute-force enumeration of entire Kronecker matrix.
    '''
    A_kron = reduce(np.kron, matrices)
    return A_kron @ y.ravel()

def mv_einstein(matrices, y):
    '''
    Use Einstein summation convention to iteratively
    contract along secondary axes and implement Kronecker 
    matrix-vector product
    '''
    p = len(matrices)
    
    if p > 13:
        raise ValueError('There aren\'t enough letters in the alphabet for this operation :(')
    
    letter_pairs = [letters[2*i]+letters[2*i+1] for i in range(p)]
    matrix_string = ','.join(letter_pairs)
    vec_in_string, vec_out_string = [''.join(s) for s in zip(*letter_pairs)]
    string_spec = f'{matrix_string},{vec_in_string}->{vec_out_string}'
    
    return np.einsum(string_spec, *matrices, y, optimize='greedy').ravel(), string_spec

In [8]:
ein_times  = []
kron_times = []
dimensions = []
import time
for scale in [1, 2, 3, 4, 5, 6, 7, 8]:
    sizes = [2*scale, 4*scale, 8*scale]
    matrices = [np.random.randn(n,n) for n in sizes]
    matrices = [X@X.T for X in matrices]
    y = np.random.randn(*sizes)
    
    start_kron = time.perf_counter()
    mv_kron(matrices, y)
    end_kron = time.perf_counter() - start_kron
    
    start_ein = time.perf_counter()
    _, string_spec = mv_einstein(matrices, y)
    end_ein = time.perf_counter() - start_ein
    
    ein_times  += [end_ein]
    kron_times += [end_kron]
    dimensions += [y.size]
    print('Dimension:', str(y.size).ljust(5), f'Einstein time: {end_ein:.3f} s.', f'Naive time {end_kron:.3f}s.')


Dimension: 64    Einstein time: 0.000 s. Naive time 0.000s.
Dimension: 512   Einstein time: 0.000 s. Naive time 0.008s.
Dimension: 1728  Einstein time: 0.000 s. Naive time 0.008s.
Dimension: 4096  Einstein time: 0.000 s. Naive time 0.049s.
Dimension: 8000  Einstein time: 0.010 s. Naive time 0.172s.
Dimension: 13824 Einstein time: 0.020 s. Naive time 0.478s.
Dimension: 21952 Einstein time: 0.052 s. Naive time 1.247s.
Dimension: 32768 Einstein time: 0.014 s. Naive time 4.383s.
